In [1]:
import pandas as pd
import numpy as np
import sklearn

In [13]:
data = pd.read_pickle('player_2016_2018.pkl')
data = data.dropna()
data['DK_SAL'] = pd.to_numeric(data['DK_SAL'])
data['VALUE'] = pd.to_numeric(data['VALUE'])
data['FAVORITE'] = data['FAVORITE'].astype(int)
data['SPREAD_10+'] = data['SPREAD_10+'].astype(int)
data['CAP_HIT'] = data['DK_SAL'] / 50000
data['CAP_VALUE'] = data['DraftKings_PTS'] / data['CAP_HIT']
data['magic_number'] = data['DraftKings_PTS'] * data['CAP_VALUE']

In [3]:
X = data.drop(['DK_SAL','DATE', 'PLAYER', 'TEAM', 'OPPONENT','C_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS',
       'SF_DK_PTS', 'SG_DK_PTS', 'opp_C_allowed', 'opp_PF_allowed',
       'opp_PG_allowed', 'opp_SF_allowed', 'opp_SG_allowed','DraftKings_PTS',
        'PACE', 'OEFF', 'DEFF','MINUTES','USAGE', 'VALUE','DK_PTS_PER_MIN','CAP_HIT', 'CAP_VALUE'],axis=1)
X.columns

Index(['POSITION', 'ROAD_HOME', 'DK_POS', 'MINUTES_roll3', 'MINUTES_roll5',
       'MINUTES_roll10', 'MINUTES_roll25', 'DraftKings_PTS_roll3',
       'DraftKings_PTS_roll5', 'DraftKings_PTS_roll10',
       'DraftKings_PTS_roll25', 'USAGE_roll3', 'USAGE_roll5', 'USAGE_roll10',
       'USAGE_roll25', 'REST DAYS', 'OPENING TOTAL', 'FAVORITE', 'SPREAD_10+',
       'PACE_roll3', 'PACE_roll5', 'PACE_roll10', 'PACE_roll25', 'OEFF_roll3',
       'OEFF_roll5', 'OEFF_roll10', 'OEFF_roll25', 'DEFF_roll3', 'DEFF_roll5',
       'DEFF_roll10', 'DEFF_roll25', 'opp_C_allowed_roll3',
       'opp_C_allowed_roll5', 'opp_C_allowed_roll10', 'opp_C_allowed_roll25',
       'opp_PF_allowed_roll3', 'opp_PF_allowed_roll5', 'opp_PF_allowed_roll10',
       'opp_PF_allowed_roll25', 'opp_PG_allowed_roll3', 'opp_PG_allowed_roll5',
       'opp_PG_allowed_roll10', 'opp_PG_allowed_roll25',
       'opp_SF_allowed_roll3', 'opp_SF_allowed_roll5', 'opp_SF_allowed_roll10',
       'opp_SF_allowed_roll25', 'opp_SG_allowed_rol

In [4]:
data = pd.get_dummies(data)
X = pd.get_dummies(X)

In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale


/Users/willhall/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/willhall/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
X_train, X_test = train_test_split(data, test_size=0.2,shuffle=False)
 


In [7]:
y_train = X_train.pop('magic_number')
y_test = X_test.pop('magic_number')
y = X.pop('magic_number')

In [8]:
X_train_scaled = pd.DataFrame(scale(X_train))
X_test_scaled = pd.DataFrame(scale(X_test))
X_scaled = pd.DataFrame(scale(X))

In [9]:
knn = KNeighborsRegressor(n_neighbors=154,weights='distance',n_jobs=-1)



In [10]:
knn.fit(X_train_scaled,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=154, p=2,
          weights='distance')

In [11]:
y_oob_knn = knn.predict(X_test_scaled)

In [14]:
df = data[['DATE','PLAYER','TEAM','DK_POS','DK_SAL','DraftKings_PTS','magic_number']]

In [15]:
result = df.iloc[40150:,:]

In [16]:
print(y_oob_knn.shape)
print(result.shape)

(10038,)
(10038, 7)


In [17]:
result = result.assign(magic_number_PRED = y_oob_knn)



In [18]:
result.to_pickle('Prediction_master_number_attempt_knn_154.pkl')

In [ ]:
k_range = [153,154,155,156,157]
k_scores = []
scoring = ['neg_mean_absolute_error']
for k in k_range:
    knn = KNeighborsRegressor(n_neighbors=k, n_jobs=-1)
    scores = cross_val_score(knn,X,y,cv=5,scoring = 'neg_mean_absolute_error', n_jobs=-1)
    k_scores.append(scores.mean())
print(k_scores)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')

In [ ]:
k_range = list(range(1,31))
print(k_range)

In [ ]:
param_grid = dict(n_neighbors=k_range)
print(param_grid)

In [ ]:
grid = GridSearchCV(knn, param_grid, cv=10, scoring = 'accuracy',n_jobs=-1)

In [ ]:
grid.fit(X,y)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=20,n_jobs=-1)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=155,n_jobs=-1)

In [ ]:
knn.fit(X_train_scaled,y_train)

In [ ]:
y_oob_knn = knn.predict(X_test_scaled)

In [ ]:
df = data[['DATE','PLAYER', 'TEAM','DK_POS','DK_SAL','MINUTES','DraftKings_PTS']]

In [ ]:
result = df.iloc[40150:,:]

In [ ]:
print(y_oob_knn.shape)
print(result.shape)

In [ ]:
# result = result.assign(magic_number_knn_20 = y_oob_knn)

result = result.assign(magic_number_knn_155 = y_oob_knn)



In [ ]:
result.to_pickle('Prediction_master_number_attempt_knn_20.pkl')

In [ ]:
result.to_pickle('Prediction_master_number_attempt_knn_155.pkl')